# Демонстрация работы нейронной сети

## 1. Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')

from src.constants import (
    NUM_DAYS,
    NUM_EMPLOYEES,
    OUTPUT_FILE,
)
from src.schedule_generator import ScheduleGenerator
from src.schedule_validator import ScheduleValidator
from src.schedule_model import ScheduleModel

## 2. Генерация расписания

In [26]:
schedule_generator = ScheduleGenerator(num_employees=NUM_EMPLOYEES, num_days=NUM_DAYS)
generated_schedule = schedule_generator.generate_schedule(shuffle_rows=True)
schedule_df = pd.DataFrame(
    generated_schedule,
    columns=[f'Day_{i+1}' for i in range(NUM_DAYS)]
)

# Сохранение расписания
schedule_df.to_csv(OUTPUT_FILE, index=False)

## 3. Подготовка данных для обучения

В этом разделе шум добавляется к исходному расписанию для создания обучающих данных. Шум помогает:
1. Сделать модель более устойчивой
2. Улучшить обобщающую способность модели
3. Избежать переобучения
4. Симулировать реальные ситуации, когда данные могут содержать неточности

In [ ]:
schedule = schedule_df.values
noisy_schedule = ScheduleGenerator.add_noise_to_schedule(schedule, noise_level=0.50)

# Преобразование данных
x = noisy_schedule.reshape(noisy_schedule.shape[0], -1)
y = schedule.reshape(schedule.shape[0], -1)    

# Разделение на тренировочные и тестовые данные
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

## 4. Создание и обучение модели

In [ ]:
output_size = x_train.shape[1] * 3 # 180 * 3
model = ScheduleModel(input_size=x_train.shape[1], output_size=output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(x_train) # (240, 180, 3)
    # Меняю форму y_train на (batch_size, 180)
    loss = criterion(outputs.view(-1, 3), y_train.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Сохранение модели
torch.save(model.state_dict(), '../shift_scheduler_model.pth')

Epoch 1/1000, Loss: 1.1558618545532227
Epoch 2/1000, Loss: 1.1386687755584717
Epoch 3/1000, Loss: 1.1258410215377808
Epoch 4/1000, Loss: 1.1170854568481445
Epoch 5/1000, Loss: 1.110834002494812
Epoch 6/1000, Loss: 1.106832504272461
Epoch 7/1000, Loss: 1.1048458814620972
Epoch 8/1000, Loss: 1.103739619255066
Epoch 9/1000, Loss: 1.103070616722107
Epoch 10/1000, Loss: 1.1024713516235352
Epoch 11/1000, Loss: 1.1019948720932007
Epoch 12/1000, Loss: 1.1017934083938599
Epoch 13/1000, Loss: 1.1010074615478516
Epoch 14/1000, Loss: 1.1002918481826782
Epoch 15/1000, Loss: 1.0997884273529053
Epoch 16/1000, Loss: 1.09956693649292
Epoch 17/1000, Loss: 1.097865343093872
Epoch 18/1000, Loss: 1.0971821546554565
Epoch 19/1000, Loss: 1.0967732667922974
Epoch 20/1000, Loss: 1.0951013565063477
Epoch 21/1000, Loss: 1.0949980020523071
Epoch 22/1000, Loss: 1.0949145555496216
Epoch 23/1000, Loss: 1.0931243896484375
Epoch 24/1000, Loss: 1.0914868116378784
Epoch 25/1000, Loss: 1.090446949005127
Epoch 26/1000, Lo

## 5. Оценка модели на тестовых данных

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(x_test) # (60, 180, 3)
    y_pred = torch.argmax(y_pred, dim=2).numpy() # (60, 180)

## 6. Вычисление метрик

In [ ]:
y_pred_flat = y_pred.flatten() # (10800,)
y_test_flat = y_test.flatten() # (10800,)

accuracy = accuracy_score(y_test_flat, y_pred_flat)
precision = precision_score(y_test_flat, y_pred_flat, average='weighted')
recall = recall_score(y_test_flat, y_pred_flat, average='weighted')
f1 = f1_score(y_test_flat, y_pred_flat, average='weighted')

print("Метрики на тестовых данных:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Метрики на тестовых данных:
Accuracy: 0.9687
Precision: 0.9688
Recall: 0.9687
F1 Score: 0.9687


## 7. Проверка расписания

In [22]:
# Проверка расписания
validator = ScheduleValidator(schedule)
validation_results = validator.validate_all_rules()

for rule, passed in validation_results.items():
    status = True if passed else False
    print(f"{rule}: {status}")

# Дозапись в файл с расписанием
if all(validation_results.values()): # Если все проверки пройдены успешно
    schedule_df.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)

Минимум два выходных на неделе: True
Выходные идут подряд: False
Минимум 2 сотрудника на смене: True
Нет утренней после вечерней: True
Смена утро/вечер каждую неделю: True
Не более 5 рабочих дней подряд: True


## 8. Демонстрация предсказанных значений

In [34]:
print(y_pred[:2, :])

[[2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 2 2 2 2 2 2 0 0 1 1 1 1 1
  0 2 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 1 1 1 1 1 1 0 0 2 2 2 2 2 0
  1 1 1 1 1 1 0 2 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0
  2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1
  1 1 1 1 0 2 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 1 1 1 1 1 1 0 2 2 2]
 [0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0
  1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2
  2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1
  1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2
  2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1 1]]


## 9. Размер модели

In [ ]:
!ls -lh ../shift_scheduler_model.pth # 49 КБ

-rw-r--r--  1 plamya  staff    49K Mar 25 08:04 ../shift_scheduler_model.pth
